In [1]:
import pandas as pd
import numpy as np
import boto3

client = boto3.client('s3')

bucket='asdi-hackathon'

file_key1 = 'population-data/popcsv/longitude0.csv'
file_key2 = 'population-data/popcsv/longitudeNeg10.csv'

csv_obj1 = client.get_object(Bucket=bucket, Key=file_key1)
csv_obj2 = client.get_object(Bucket=bucket, Key=file_key1)

obj1 = client.get_object(Bucket=bucket, Key=file_key1)
obj2 = client.get_object(Bucket=bucket, Key=file_key2)

df1 = pd.read_csv(obj1['Body'])
df2 = pd.read_csv(obj2['Body'])

#df = pd.read_csv('/Users/joshua.grefte/Projects/ASDI/Data/so2_sentinel_01-16june.csv')

In [2]:
df1.head()

,Unnamed: 0,latitude,longitude,population
0,1308823,51.560139,0.227083,5.40624
1,1308824,51.560694,0.226250,5.40624
2,1308825,51.560417,0.226250,5.40624
3,1308826,51.560417,0.228472,5.40624
4,1308827,51.559861,0.227361,5.40624


In [3]:
df2.head()

,Unnamed: 0,latitude,longitude,population
0,126943,51.620417,-0.288472,46.080284
1,126944,51.620417,-0.289306,46.080284
2,126945,51.620417,-0.289028,46.080284
3,126946,51.620139,-0.288194,46.080284
4,126947,51.620139,-0.288750,46.080284


In [9]:
df = df1.append(df2)
df = df.drop('Unnamed: 0', axis = 1)
df

/var/folders/70/yx3rlp812k545ncgsfplvcbc0000gn/T/ipykernel_14194/2697515025.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2)


,latitude,longitude,population
0,51.560139,0.227083,5.406240
1,51.560694,0.226250,5.406240
2,51.560417,0.226250,5.406240
3,51.560417,0.228472,5.406240
4,51.559861,0.227361,5.406240
...,...,...,...
1148462,51.554583,-0.208472,33.506246
1148463,51.553750,-0.207639,33.506246
1148464,51.554028,-0.207639,33.506246
1148465,51.554861,-0.208750,33.506246


In [14]:
print(max(df['latitude']))
min(df['latitude'])

51.69291666666667


51.26513888888889

In [15]:
print(max(df['longitude']))
min(df['longitude'])

0.56125


-0.7465277777777778

In [17]:
nrow_before = len(df)
df_filter = df[(df['latitude'] > 51.239405) & (df['latitude'] < 51.737184)]
df_filter = df_filter[(df_filter['longitude'] > -0.625211) & (df_filter['longitude'] < 0.328289)]
nrow_after = len(df_filter)
print('Number of rows removed:', nrow_before - nrow_after)
df = df_filter

Number of rows removed: 264834


In [18]:
df

,latitude,longitude,population
0,51.560139,0.227083,5.406240
1,51.560694,0.226250,5.406240
2,51.560417,0.226250,5.406240
3,51.560417,0.228472,5.406240
4,51.559861,0.227361,5.406240
...,...,...,...
1148462,51.554583,-0.208472,33.506246
1148463,51.553750,-0.207639,33.506246
1148464,51.554028,-0.207639,33.506246
1148465,51.554861,-0.208750,33.506246


In [19]:
#haversine requires latitude and longitude in radians
import math
df['latitude'] = df['latitude'].apply(math.radians)
df['longitude'] = df['longitude'].apply(math.radians)

In [20]:
from sklearn.neighbors import KNeighborsRegressor

#use all of data in train
X_train = df[['latitude', 'longitude']]
y_train = df['population'].ravel()

#use best k value
k = 50
model = KNeighborsRegressor(n_neighbors=k, weights = 'distance', algorithm = 'brute', metric = 'haversine', n_jobs = -1)
model.fit(X_train, y_train)

KNeighborsRegressor(algorithm='brute', metric='haversine', n_jobs=-1,
                    n_neighbors=50, weights='distance')

In [21]:
#pickle final model
import pickle
filename = 'popdensity_model.pkl'
model = model
pickle.dump(model, open(filename, 'wb'))

In [6]:
!pip3 install tqdm --quiet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut

df_train, df_test = train_test_split(df, random_state = 0, test_size = 0.01)

X_train = df_train[['latitude', 'longitude']]
y_train = df_train['population'].ravel()
X_test = df_test[['latitude', 'longitude']]
y_test = df_test['population'].ravel()

#X = df[['latitude', 'longitude']]
#y = df['population'].ravel()

k_range = range(1, 200)
k_scores = []

#uses 1 data point taken out as validation set with rest used for "training"
cv = LeaveOneOut()

from tqdm import tqdm
for k in tqdm(k_range):
    model = KNeighborsRegressor(n_neighbors=k, weights = 'distance', algorithm = 'kd_tree', metric = 'haversine', n_jobs = -1)
    loss = abs(cross_val_score(model, X_train, y_train, cv=cv, scoring='neg_root_mean_squared_error', n_jobs = -1))
    k_scores.append(loss.mean())

model.fit(X_train, y_train)

  0%|          | 0/199 [05:23<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(k_range, k_scores)
plt.xlabel('Value of K for KNN')
plt.ylabel('Cross-Validated RMSE')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
results = pd.DataFrame(dtype = 'object')
results['y_test'] = y_test
results['y_pred'] = y_pred
results['difference'] = y_test - y_pred
results['abs_diff'] = abs(y_test - y_pred)
results.head()

In [ ]:
results.describe()

In [ ]:
results['abs_diff'].plot.bar()

In [ ]:
df['population'].describe()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, random_state = 0, test_size = 0.01)

X_train = df_train[['latitude', 'longitude']]
y_train = df_train['population'].ravel()
X_test = df_test[['latitude', 'longitude']]
y_test = df_test['population'].ravel()

#X = df[['latitude', 'longitude']]
#y = df['population'].ravel()

cv = LeaveOneOut()

parameters = {'n_neighbors': np.arange(start=1, stop=200, step=2)}
grid_search = GridSearchCV(KNeighborsRegressor(weights = 'distance', algorithm = 'kd_tree', metric = 'haversine'), 
                           param_grid = parameters, 
                           scoring='neg_root_mean_squared_error', 
                           n_jobs = -1,
                           cv = cv,
                           error_score = 'raise')
grid_search.fit(X_train, y_train)

print('best parameter(s): ', grid_search.best_params_)
print('best score(s): ', grid_search.best_score_)

In [ ]:
#use all of data in train
X_train = df[['latitude', 'longitude']]
y_train = df['population'].ravel()

#use best k value
k = grid_search.best_params_['n_neighbors']
model = KNeighborsRegressor(n_neighbors=k, weights = 'distance', algorithm = 'brute', metric = 'haversine', n_jobs = -1)
model.fit(X_train, y_train)

In [ ]:
#pickle final model
import pickle
filename = 'popdensity_model.pkl'
model = model
pickle.dump(model, open(filename, 'wb'))